# Tuning Fashion MNIST

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Flatten, Dense
from keras.models import Sequential
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import random
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator


2025-02-20 14:57:16.610316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740081436.834253  307888 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740081436.902037  307888 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Load MNIST dataset
mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 28, 28, 1).astype('float32') / 255.0
x_test = x_test.reshape(10000, 28, 28, 1).astype('float32') / 255.0

Here are the names of the 10 classes

In [3]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

Things to do:
*  Plot a histogram of the class labels to determine balance
*  Plot some random entries from the dataset and their class name
*  Create a DNN based on the MNIST one we made. Use any topology and activation functions you want.
*  When you 'compile' the model, include `metrics=['accuracy']`
*  Train the model
*  Plot the accuracies using the plot code below
*  Evaluate the model on the **test** set and report the result
*  Repeat the last 5 steps -- try to get the best test accuracy
*  Print a confusion matrix (10x10) for the test set
*  Make a grid showing a selection of the misclassified samples, along with their label (probably need to google this)

## Fine Tuning

In [4]:
import keras_tuner as kt

In [5]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value = 2, max_value = 8, default = 3)
    n_neurons = hp.Int("n_neurons", min_value = 32, max_value = 192)
    n_dropouts = hp.Int("n_dropouts", min_value = 2, max_value = 15)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input([28,28]))
    model.add(tf.keras.layers.Flatten())
    for _ in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="selu"))
        model.add(tf.keras.layers.Dropout(n_dropouts/100))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
    return model

In [6]:
random_search_tuner = kt.RandomSearch(
    build_model, objective='val_accuracy', max_trials=50, overwrite=True,
    directory="fashion_mnist", project_name="search", seed=42)

I0000 00:00:1740081467.976047  307888 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2256 MB memory:  -> device: 0, name: NVIDIA T500, pci bus id: 0000:01:00.0, compute capability: 7.5


In [8]:
random_search_tuner.search(x_train, y_train, epochs=15, validation_split=0.1, batch_size=1024)

Trial 35 Complete [00h 00m 39s]
val_accuracy: 0.8840000033378601

Best val_accuracy So Far: 0.8878333568572998
Total elapsed time: 00h 39m 00s

Search: Running Trial #36

Value             |Best Value So Far |Hyperparameter
6                 |6                 |n_hidden
96                |178               |n_neurons
2                 |6                 |n_dropouts

Epoch 1/15


KeyboardInterrupt: 

In [ ]:
random_search_tuner.get_best_hyperparameters().